In [4]:
import json
import pandas as pd
from tqdm import tqdm
import nltk
# nltk.download("all")

from nltk import sent_tokenize
from rank_bm25 import BM25Okapi
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer

d:\level2-mrc-nlp-08\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.auto.tokenization_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
No module named 'numpy.rec'

# Load Dataset
- KorQuad v1 데이터셋 로드
- 원본 데이터도 불러와 학습 데이터에 추가

In [5]:
dataset = load_dataset('KorQuAD/squad_kor_v1')
dataset_raw = load_from_disk("../resources/train_dataset")

Generating validation split: 100%|██████████| 5774/5774 [00:00<00:00, 409349.10 examples/s]


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [7]:
dataset['train'][0]

{'id': '6566495-0-0',
 'title': '파우스트_서곡',
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': {'text': ['교향곡'], 'answer_start': [54]}}

In [8]:
dataset_raw

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [9]:
dataset_raw['train'][0]

{'title': '미국 상원',
 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05',
 'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?',
 'id': 'mrc-1-000067',
 'answers': {'answer_start'

# Chunk Contexts
- nltk의 sent_tokenize를 이용해서 contect를 chunk 단위로 분할
- langchain의 text_splitter와 비교 필요

In [10]:
def chunk_context(context, num_sents, overlaps):
    """
    Arguments:
        context
        num_sents: 한 chunk 당 문장의 수
        context_path: 연속된 chunk 간 겹치는 문장의 수
    """
    chunks = []
    start, end = 0, num_sents
    
    total_sents = sent_tokenize(context)

    while start < len(total_sents):
        chunk = total_sents[start:end]
        chunks.append(' '.join(chunk))
        
        start += (num_sents - overlaps)
        end = start + num_sents

    return chunks

In [11]:
sample_ctx = dataset['train'][0]['context']

chunks = chunk_context(sample_ctx, 3, 1)
chunks

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\user/nltk_data'
    - 'd:\\level2-mrc-nlp-08\\.venv\\nltk_data'
    - 'd:\\level2-mrc-nlp-08\\.venv\\share\\nltk_data'
    - 'd:\\level2-mrc-nlp-08\\.venv\\lib\\nltk_data'
    - 'C:\\Users\\user\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


# Training Templates
- DPR 모델 학습을 위한 템플릿
- TEMPATE_ALL 의 positive 키의 리스트에 TEMPLATE_TEXT가 요소로 들어감

In [10]:
TEMPLATE_ALL = {
    "question": "",
	"answers": [],
	"positive": [],
  }

TEMPLATE_TEXT = {
    "title": "",
    "text": "",
}

- fill_templates 함수는 여러 개의 chunk 중 단답형 정답을 포함하는 chunk만 템플릿 구성
- 제목(title)이 없으면 그대로 "" 로 두면 됨

In [11]:
def fill_templates(chunks, title, question, answer):
    positives = []
    for chunk in chunks:
        template_instance1 = TEMPLATE_TEXT.copy()
        template_instance1['title'] = title
        template_instance1['text'] = chunk
    
        if answer in chunk:
            positives.append(template_instance1.copy())

    template_instance2 = TEMPLATE_ALL.copy()
    template_instance2['question'] = question
    template_instance2['answers'] = [answer]
    template_instance2['positive'] = positives
    
    return template_instance2

In [12]:
dataset['train'][0]['answers']['text'][0]

'교향곡'

In [13]:
fill_templates(chunks, 
               dataset['train'][0]['title'], 
               dataset['train'][0]['question'], 
               dataset['train'][0]['answers']['text'][0])

{'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': ['교향곡'],
 'positive': [{'title': '파우스트_서곡',
   'text': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.'},
  {'title': '파우스트_서곡',
   'text': '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.'},
  {'title': '파우스트_서곡',
   'text': '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.'}]}

# DPR 학습 데이터 생성

In [14]:
def construct_dpr_dataset(dataset, num_sents, overlaps):
    dpr_dataset = []
    for idx in tqdm(range(len(dataset))):
        sample = dataset[idx]
        
        title = sample['title']
        question = sample['question']
        answer = sample['answers']['text'][0]
        chunks = chunk_context(sample['context'], num_sents, overlaps)
        
        output = fill_templates(chunks, title, question, answer)
        dpr_dataset.append(output)
    
    return dpr_dataset

In [14]:
dpr_korquad_dataset = construct_dpr_dataset(dataset['train'], 3, 1)

100%|██████████| 60407/60407 [00:18<00:00, 3210.16it/s]


In [15]:
dpr_korquad_dataset[0]

{'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': ['교향곡'],
 'positive': [{'title': '파우스트_서곡',
   'text': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.'},
  {'title': '파우스트_서곡',
   'text': '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.'},
  {'title': '파우스트_서곡',
   'text': '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.'}]}

In [15]:
dpr_raw_dataset = construct_dpr_dataset(dataset_raw['train'], 3, 1)

  0%|          | 0/3952 [00:00<?, ?it/s]

100%|██████████| 3952/3952 [00:02<00:00, 1323.14it/s]


In [17]:
dpr_raw_dataset[0]

{'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?',
 'answers': ['하원'],
 'positive': [{'title': '미국 상원',
   'text': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다.'},
  {'title': '미국 상원',
   'text': '임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다.'},
  {'title': '미국 상원',
   'text': '즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다.'},
  {'title': '미국 상원',
   'text': '그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기

In [18]:
dpr_dataset = dpr_korquad_dataset + dpr_raw_dataset

In [19]:
len(dpr_dataset)

64359

In [16]:
dpr_dataset_train = dpr_raw_dataset

In [17]:
dpr_dataset_valid = construct_dpr_dataset(dataset_raw['validation'], 3, 1)
len(dpr_dataset_valid)

100%|██████████| 240/240 [00:00<00:00, 2184.12it/s]


240

# Set Index
- 데이터 셋의 각 텍스트에 고유 인덱스 부여
- 데이터 셋에 동일한 텍스트가 여러 개 있을 경우 모두 동일한 인덱스를 부여

In [17]:
def set_index_to_text(dataset):
    text_2_index = {}
    text_2_title = {}
    current_idx = 0  # Initialize the starting index

    for sample in dataset:
        pos_text = [pos['text'] for pos in sample['positive']]
        pos_title = [pos['title'] for pos in sample['positive']]
    
        all_idx = []
        for text, title in zip(pos_text, pos_title):
            if text not in text_2_index:
                text_2_index[text] = current_idx 
                text_2_title[text] = title  # Map the text to its title
                current_idx += 1

        for pos in sample['positive']:
            pos['idx'] = text_2_index[pos['text']]
            all_idx.append(text_2_index[pos['text']])
        
        sample['answer_idx'] = all_idx

    return dataset, text_2_index, text_2_title

In [21]:
dpr_dataset, text_2_index, text_2_title = set_index_to_text(dpr_dataset)

In [18]:
dpr_dataset_train, text_2_index_train, text_2_title_train = set_index_to_text(dpr_dataset_train)

In [19]:
dpr_dataset_valid, text_2_index_valid, text_2_title_valid = set_index_to_text(dpr_dataset_valid)

# Add Hard Negative
- get_hard_negative 함수를 이용해 '정답이 아니지만 정답처럼 보이는 텍스트(Hard Negative)' 찾기
- BM25 점수가 가장 높은 n 개를 추출한 후 그 중에서 가장 높은 점수의 텍스트가 차례로 Hard Negative 샘플로 선택
- KorQuad의 경우 여러 질문이 동일한 텍스트에 연결된 경우가 많아 False Negative를 방지하기 위해 동일한 텍스트는 모두 필터링
- 필터링의 결과로 n 개의 텍스트에서 Hard Negative 샘플을 정해진 수만큼 찾지 못한 경우 기존의 것을 복제해 사용
- positive의 텍스트 개수와 hard_neg의 텍스트 개수는 일치해야 함

In [19]:
from kiwipiepy import Kiwi
kiwi=Kiwi()

In [20]:
def get_hard_negative(dataset, text_2_index, text_2_title, n=30):
    corpus = list(text_2_index.keys())
    
    tokenized_corpus = []
    for context in corpus:
        morph_context = kiwi.tokenize(context)
        morph_context_form = [x.form for x in morph_context]
        tokenized_corpus.append(morph_context_form)
    
    bm25 = BM25Okapi(tokenized_corpus)

    for sample in tqdm(dataset, desc="Processing samples"):
        question = sample['question']
        morph_question = kiwi.tokenize(question)
        morph_question_form = [x.form for x in morph_question]
        positive = sample['positive']

        top_n = bm25.get_top_n(morph_question_form, corpus, n=n)  # Increase n to have more candidates
        
        hard_neg = []
        positive_idx_set = set(sample['answer_idx'])
        for doc in top_n:
            if text_2_index[doc] not in positive_idx_set:
                hard_neg.append({'title': text_2_title[doc],
                                 'text': doc,
                                 'idx': text_2_index[doc]})
            
            if len(hard_neg) >= len(positive):
                break

        if len(hard_neg) == 0:
            print(f"Skipping sample due to lack of hard negatives: {question}")
            continue
        
        # If not enough hard negatives, repeat existing ones to match the number of positives
        if len(hard_neg) < len(positive):
            print(f"Warning: Not enough hard negatives for question: {question}. Duplicating existing hard negatives.")
            while len(hard_neg) < len(positive):
                hard_neg.extend(hard_neg[:len(positive) - len(hard_neg)])
        
        sample['hard_neg'] = hard_neg[:len(positive)]  # Trim to ensure exact number matches positive samples
        
    return dataset

In [24]:
dpr_dataset = get_hard_negative(dpr_dataset, text_2_index, text_2_title, n=30)

Processing samples: 100%|██████████| 64359/64359 [4:12:42<00:00,  4.24it/s]  


In [25]:
dpr_dataset[0]

{'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': ['교향곡'],
 'positive': [{'title': '파우스트_서곡',
   'text': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.',
   'idx': 0},
  {'title': '파우스트_서곡',
   'text': '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.',
   'idx': 1},
  {'title': '파우스트_서곡',
   'text': '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.',
   'idx': 2}],
 'an

In [26]:
file_path = 'dpr_train_v2.json'

In [27]:
with open(file_path, 'w') as outfile:
    json.dump(dpr_dataset, outfile, indent=4)

In [22]:
dpr_dataset_train = get_hard_negative(dpr_dataset_train, text_2_index_train, text_2_title_train, n=30)

Processing samples: 100%|██████████| 3952/3952 [03:05<00:00, 21.35it/s]


In [23]:
file_path = 'dpr_raw_train_v2.json'
with open(file_path, 'w') as outfile:
    json.dump(dpr_dataset_train, outfile, indent=4)

In [22]:
dpr_dataset_valid = get_hard_negative(dpr_dataset_valid, text_2_index_valid, text_2_title_valid, n=30)

Processing samples: 100%|██████████| 240/240 [00:00<00:00, 401.74it/s]


In [24]:
file_path = 'dpr_valid_v2.json'
with open(file_path, 'w') as outfile:
    json.dump(dpr_dataset_valid, outfile, indent=4)